In [ ]:
import pandas as pd

In [ ]:
# Liste des algorithmes à inclure
algorithms = [
    'BIPOP-CMA-ES', 'CMA-CSA_Atamna', 'CMAES-APOP-KMA_Nguyen',
    'DE-BFGS_voglis_noiseless', 'a-CMA-ES', 'ad-CMA-ES_Gissler',
    'adm-CMA-ES_Gissler', 'dm-CMA-ES_Gissler', 's-CMA-ES_Gissler',
    'sd-CMA-ES_Gissler'
]

In [ ]:
# Groupes de fonctions
function_groups = {
    '1-5': list(range(1, 6)),
    '6-9': list(range(6, 10)),
    '10-14': list(range(10, 15)),
    '15-19': list(range(15, 20)),
    '20-24': list(range(20, 25))
}

In [ ]:
# Lecture du fichier CSV
df = pd.read_csv('optimization_data_relert.csv')

In [ ]:
# Filtrer les algorithmes
df_filtered = df[df['Algorithm'].isin(algorithms)].copy()

In [ ]:
# Ajouter le groupe de fonctions
def assign_function_group(func):
    for group_name, functions in function_groups.items():
        if func in functions:
            return group_name
    return None

In [ ]:
df_filtered['function_group'] = df_filtered['Function'].apply(assign_function_group)
df_filtered = df_filtered.dropna(subset=['function_group'])

In [ ]:
# Calcul de la moyenne par Dimension, function_group et Algorithm
result = df_filtered.groupby(['Dimension', 'function_group', 'Algorithm'])['RELERT'].mean().reset_index()

In [ ]:
# Pivot: algorithmes en colonnes
pivot_result = result.pivot(index=['Dimension', 'function_group'], columns='Algorithm', values='RELERT')

In [ ]:
# Ajouter la ligne "ALL" pour chaque Dimension
all_per_dimension = (
    pivot_result
    .groupby(level=0)  # par Dimension uniquement
    .mean()
    .assign(function_group='ALL')
    .set_index('function_group', append=True)
)

In [ ]:
# Ajouter la ligne "ALL/ALL" pour tout
all_overall = pd.DataFrame(
    pivot_result.mean().to_dict(),
    index=[('ALL', 'ALL')]
)

In [ ]:
# Concaténer les résultats
final_result = pd.concat([pivot_result, all_per_dimension, all_overall])
final_result = final_result.sort_index()

In [ ]:
# Exporter en Excel
try:
    final_result.to_excel('RELERt_table_with_ALL.xlsx')
    print("Tableau final enregistré dans 'RELERt_table_with_ALL.xlsx'")
except PermissionError:
    print("Fichier ouvert ? Enregistrement sous un autre nom...")
    final_result.to_excel('RELERt_table_with_ALL_alt.xlsx')
    print("Enregistré dans 'RELERt_table_with_ALL_alt.xlsx'")